# Data Ingestion into raw folders

## support_logs

In [ ]:
import os
import boto3
from datetime import datetime, timedelta

# ---------- CONFIG ----------
LOGS_FOLDER = "day-wise-logs-data/"
DATE_TRACKER_FILE = "log_date_tracker.txt"

S3_BUCKET = "my-careplus-data" # this should match the exact bucket name you have setup in AWS S2
S3_PREFIX = "support-logs/raw/"

from dotenv import load_dotenv
load_dotenv()

os.getenv("REGION")

In [ ]:
import os

AWS_CONFIG = {
    "aws_access_key_id": os.getenv("AWS_ACCESS_KEY"),
    "aws_secret_access_key": os.getenv("SECRET_KEY"),
    "region_name": os.getenv("REGION")
}

In [ ]:
# ---------- UTILITY FUNCTIONS ----------
def read_last_date(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return f.read().strip()
    return "2025-06-30"

def update_last_date(file_path, new_date):
    with open(file_path, 'w') as f:
        f.write(new_date)

def get_next_date(last_date_str):
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
    next_date = last_date + timedelta(days=1)
    return next_date.strftime("%Y-%m-%d")


def upload_log_file_to_s3(file_path, bucket, key):
    s3 = boto3.client('s3', **AWS_CONFIG)
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    s3.put_object(Bucket=bucket, Key=key, Body=content)
    print(f"✅ Uploaded log file to s3://{bucket}/{key}")


# ---------- MAIN INGESTION LOGIC ----------
def run_log_ingestion():
    last_date = read_last_date(DATE_TRACKER_FILE)
    next_date = get_next_date(last_date)
    print(last_date, next_date)

    file_name = f"support_logs_{next_date}.log"
    log_file_full_path = os.path.join(LOGS_FOLDER, file_name)
    print(log_file_full_path)
    

    s3_key = f"{S3_PREFIX}support_logs_{next_date}.log"
    upload_log_file_to_s3(log_file_full_path, S3_BUCKET, s3_key)
    update_last_date(DATE_TRACKER_FILE, next_date)
    print(f"📅 Updated tracker to {next_date}")

In [ ]:
# ---------- RUN ----------
if __name__ == "__main__":
    run_log_ingestion()

## support_tickets

In [ ]:
import os
import pandas as pd
import boto3
from io import StringIO
from sqlalchemy import create_engine
from datetime import datetime, timedelta

from dotenv import load_dotenv

load_dotenv()

In [ ]:
# ---------- CONFIG ----------
db_config = {
    "host": "localhost",
    "port": "3306",
    "user": "root",  # change
    "password": "#####", # change
    "database": "careplus_support_db"
}

S3_BUCKET = "my-careplus-data" 
S3_PREFIX = "support-tickets/raw/"  
DATE_TRACKER_FILE = "date_tracker.txt"

import os

AWS_CONFIG = {
    "aws_access_key_id": os.getenv("AWS_ACCESS_KEY"),
    "aws_secret_access_key": os.getenv("SECRET_KEY"),
    "region_name": os.getenv("REGION")
}

In [ ]:
# ---------- UTILITY FUNCTIONS ----------
def get_engine(config):
    return create_engine(f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

def upload_to_s3(df, bucket, key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3 = boto3.client('s3', **AWS_CONFIG)
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket}/{key}")

def read_last_date(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return f.read().strip()
    return "2025-06-30"  # Starting point before 1st July

def update_last_date(file_path, new_date):
    with open(file_path, 'w') as f:
        f.write(new_date)

def get_next_date(last_date_str):
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
    next_date = last_date + timedelta(days=1)
    return next_date.strftime("%Y-%m-%d")

# ---------- MAIN INGESTION LOGIC ----------
def run_ingestion():
    engine = get_engine(db_config)
    last_date = read_last_date(DATE_TRACKER_FILE)
    next_date = get_next_date(last_date)

    # Query only that day’s data
    query = f"""
        SELECT * FROM support_tickets
        WHERE DATE(created_at) = '{next_date}';
    """
    df = pd.read_sql(query, engine)
    print(df.shape)
    print(df.head())

    if df.empty:
        print(f"⚠️ No data found for {next_date}. Skipping upload.")
        return

    # Upload to S3
    s3_key = f"{S3_PREFIX}support_tickets_{next_date}.csv"
    upload_to_s3(df, S3_BUCKET, s3_key)

    # Update date tracker
    update_last_date(DATE_TRACKER_FILE, next_date)
    print(f"📅 Updated tracker to {next_date}")

# Run
if __name__ == "__main__":
    run_ingestion()